# Spark

Last updated 17:15 20161108 word count S-7
    * word count by line
    * 빅데이터 Giga급 서버에 준비
    
## 목적

* Spark를 사용하여 데이터를 변환할 수 있다.
* Spark를 사용하여 통계분석을 할 수 있다.
* Spark를 사용하여 추천, 분류, 예측을 분석할 수 있다.

## 문제

* 문제 S-1: standalone cluster 구성
* 문제 S-2: Hello Spark
* 문제 S-3: Hello RDD
    * fivethirtyeight -> data_home=os.path.join(home,"Code/git/else/data")
* 문제 S-4: RDD word count
    * feature extraction - word vector
    * stopwords (nltk.corpus.stopwords)
* 문제 S-5: RDD from url
    * .toDF (rdd -> dataframe)
    * 문제 rdd babynames
* 문제 S-6: Spark SQL - SqlContext
* 문제 S-7: Spark Data frame
    * 문제 Spark SQL json
    * 문제 spark sql uber csv
* 문제 S-8: Hello Statistics
* 문제 S-9: Spark data type
* 문제 libsvm format
* 문제 S-10: Spark MLib Clustering: twitter _my.json, tf-idf
* 문제 S-11: Spark MLib Decision Tree
* 문제 S-12: Spark MLib Logistic regression
    * https://www.codementor.io/spark/tutorial/spark-mllib-logistic-regression
* 문제 S-13: Spark MLib movie recommendation 사례
* 문제 S-14: Spark Streaming
* 문제 S-15: GraphX
* 문제 시각화 Bokeh
http://www.blog.pythonlibrary.org/2016/07/27/python-visualization-with-bokeh/

* spark-submit (self-contained app in quick-start 참조)

* ref
    * introduction to big data with apache spark (Berkley Anthony Joseph)
    * [spark-sklearn](https://github.com/databricks/spark-sklearn)
        ```
        pip install spark-sklearn
        ```
        
        * spark-shell
        ```
        $SPARK_HOME/bin/spark-shell --packages databricks:spark-sklearn:0.2.0
        ```
        
    * 듀크대학 STA663 Statistical Computing and Computation, Spring 2016
        * https://github.com/cliburn/sta-663-2016

## S.1 설치

* 설치하려는 하둡의 버전을 선택하여, prebuilt distribution을 설치한다.
* [Spark 다운로드](https://spark.apache.org/downloads.html)
    * spark 1.6 hadoop2.6 
        ```
        tar xvfz spark-1.x.x-bin-hadoop2.x.tgz
        ```


## S.2 Spark 소개

* 버전
    * 2009년 UC Berkeley, Matei Zaharia 박사과정 하면서 개뱔
    * 2010년 BSD 라이센스 오픈소스로 전환.
    * 2013년 Apache 2.0 license로 전환
    * 현재 개발자가 Databricks를 설립해서 관리

* 왜 Spark를 배워야 하는가?
    * REPL도 가능해서 배우기 쉽다. 쉘 환경이 있어 편리하다. Standalone으로 시작할 수 있다.
    * 빅데이터를 빠르게 map reduce 할 수 있다.
    * Machine learning lib를 가지고 있다.

* 구조
    * Spark Core
    * 분산 클러스터 컴퓨팅 프레임워크로서, API를 사용해서 데이터추출, 변환, 기계학습, 그래프분석을 할 수 있다.

구분 | 구성 | 설명
-------|-------|-------
Spark engine | Spark Core |
Spark Applicaiton Frameworks | Spark SQL | DataFrames
| Spark Streaming | 
| MLlib | 머신러닝 (참조 scikit-learn)
|GraphX | 그래프 분석


    * 빅데이터를 처리하기 위해 만들어져 있고, Hadoop과 달리 메모리에서 처리하기 때문에 빠름 (pipeline). Spark는 RDD를 통해 Hadoop을 사용할 수 있다.
    * Scala로 개발되어 jvm에서 실행. 그러나 Scala, Java, Python, R 여러 언어를 섞어서 할 수 있는 환경을 제공 (polyglot)

구분 | Spark | Hadoop
-------|-------|-------
사용 목적 | 데이터 분석 | 데이터 분산 처리
파일 시스템 | 자체 파일 시스템이 없슴. hdfs, db, csv등을 사용 | hdfs
속도 | 파이프라인을 사용하므로 빠름 | 보다 느림

* batch, streaming, iterative, interactive 4가지 방식으로 실행할 수 있다.


## 문제 S-1: standalone cluster 구성

* 클러스터를 구성하지 않으면 클러스터 없이 운영 - curl로 7077, 8080확인해도 없음 -> NO cluster!

sbin디렉토리의 shell | 설명
----------|----------
start-master.sh, stop-master.sh | 마스터를 시작 (종료)
start-slaves.sh, stop-slaves.sh | 각 노드의 슬레이브를 시작 (종료)
start-all.sh, stop-all.sh | 마스터, 슬레이브를 모두 시작 (종료)

* 클러스터 종류
    * Spark-Standalone – Spark workers are registered with spark master
    * Yarn – Spark workers are registered with YARN Cluster manager.
    * Mesos – Spark workers are registered with Mesos.

*  Spark-Standalone
    * SPARK_HOME은 /home/jsl/Downloads/spark-1.6.0-bin-hadoop2.6/
    * 단계1 JAVA_HOME을 설정
        * JAVA_HOME을 설정한다.
            * automatically set symlink to java binary /usr/bin/java
            * JAVA_HOME을 설정하려면 /etc/environment에 하는 것이 좋다.
            ```
            $ echo $JAVA_HOME
            $ update-alternatives --config java
            ```

    * 단계2: master 실행
        * $SPARK_HOME/conf/spark-env.sh에 master ip설정
            ```
            SPARK_MASTER_IP=
            ```

        * 실행
            ```
            $ sh $SPARK_HOME/sbin/start-master.sh
            ```
            * spark://IPADRESS_OF_YOUR_MASTER_SYSTEM:7077
            * 기본 port는 7077 (web UI는 localhost:8080)

    * 단계3: slave 실행 (worker라고 함)
        ```
        $ sh $SPARK_HOME/sbin//start-slave.sh spark://IPADRESS_OF_YOUR_MASTER_SYSTEM:7077
        ```

* 환경 구성
    * Client: spark shell, pyspark shell
    * Cluster
        * Cluster 1:
            * Spark master / Spark worker
            * hdfs namenode / datanode
        * Cluster n:
            * Spark worker
            * hdfs datanode


## 문제 S-2: Hello Spark

### spark-submit
* spark 프로그램을 일괄 실행

### insteractive shell

* scala
    ```
    ./bin/spark-shell
    scala>
    ```

* python
    ```
    spark-1.6.0-bin-hadoop2.6/bin$ pyspark
    Welcome to
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
          /_/

    Using Python version 2.7.12 (default, Jul  1 2016 15:12:24)
    SparkContext available as sc, HiveContext available as sqlContext.
    >>> sc.version
    u'1.6.0'
    >>> text=sc.textFile("derby.log");
    >>>
    ```


* SparkContext
    * Spark서버(클러스터)에 대한 클라이언트와 같은 역할로, 반드시 있어야 한다.
    * 클러스터를 어떻게 사용할 것인지 정하는 것 -> cluster manager에서 system resource를 할당받음 (cpu, memory, machine)
    * Python의 SparkContext는 jar를 분산환경에서 사용하게 되므로 주의 (Scala, Java와 다름)
        * pyFiles에 사용할 (의존적인) 라이브러리를 넣는다 (또는 사용할 라이브러리가 없으면 빈 파일로 둔다).

    * Cannot run multiple SparkContexts at once;
        * sc가 이미 있는 경우
        ```
        from pyspark import SparkContext
        sc = SparkContext("master","my python app", sparkHome="sparkhome",pyFiles="placeholderdeps.zip")
        ```

* SparkConf
    * spark-defaults.conf와 같은 파일의 값을 읽어서 설정
    ```
    scala> sc.getConf.getOption("spark.local.dir")
    res0: Option[String] = None

    scala> sc.getConf.getOption("spark.app.name")
    res1: Option[String] = Some(Spark shell)

    scala> sc.getConf.get("spark.master")
    res2: String = local[*]
    ```

### PySpark on IPython Notebook

* ipython notebook에서 pyspark를 사용
* kernel을 만들지 않고, findspark를 사용한다.
* SPARK_HOME을 설정해서 사용한다
    * 현재 ~/Downloads/spark-1.6.0-bin-hadoop2.6
* kernel을 사용하면 아래와 같이 한다.
    ```
    from pyspark import SparkContext, SparkConf
    conf = SparkConf().setAppName("jsl").setMaster("local")
    sc = SparkContext(conf=conf)
    ```

In [8]:
import os
import findspark

home=os.getenv("HOME")
spark_home=os.path.join(home,"Downloads/spark-1.6.0-bin-hadoop2.6")
findspark.init(spark_home)

* SparkConf()에서 설정한 값을 읽어 SparkContext sc를 생성한다.
    * spark.master와 spark.app.name 필수적으로 설정해야 한다.
* standalone 실행에서의 spark master
    * local로 설정되고, 기본은 필요한만큼 쓰레드를 생성한다 (local[*])

In [9]:
import pyspark
conf=pyspark.SparkConf()
conf = pyspark.SparkConf().setAppName("myAppName")
sc = pyspark.SparkContext(conf=conf)

In [3]:
print sc

In [4]:
sc.version

u'1.6.0'

* 설정을 읽어 온다 (conf디렉토리 아래)  

In [3]:
sc.master

u'local[*]'

In [7]:
sc._conf.get("spark.jars.packages")

u'graphframes:graphframes:0.1.0-spark1.6,org.mongodb.spark:mongo-spark-connector_2.10:1.1.0'

In [4]:
sc._conf.getAll()

[(u'spark.app.name', u'myAppName'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.jars',
  u'file:/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,file:/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,file:/home/jsl/.ivy2/jars/org.mongodb_mongo-java-driver-3.2.2.jar'),
 (u'spark.jars.packages',
  u'graphframes:graphframes:0.1.0-spark1.6,org.mongodb.spark:mongo-spark-connector_2.10:1.1.0'),
 (u'spark.files',
  u'file:/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,file:/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,file:/home/jsl/.ivy2/jars/org.mongodb_mongo-java-driver-3.2.2.jar'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.submit.pyFiles',
  u'/home/jsl/.ivy2/jars/graphframes_graphframes-0.1.0-spark1.6.jar,/home/jsl/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.10-1.1.0.jar,/home/jsl/.ivy2/jars/org.mong

## 문제 S-3: Hello RDD

* RDD (Resilient Distributed Dataset)
    * Resilient - fault tolerent
    * Distributed - multiple nodes in a clusters
    * Dataset - 
* 분산 데이터 자료
* HDFS 파일을 처리할 수 있다.

### Spark를 사용하기 전, python 함수 사용하기

* map, reduce, filter
    * 함수의 인자는 2개가 필요하다 (함수, 데이터).

함수 | 설명 | 예
-------|-------|-------
map() | 각 데이터 요소에 함수를 적용해서 list를 반환 | map(fn,data)
filter() | 각 데이터 요소에 함수의 결과 True를 선택해서 반환 | filter(fn, data)
reduce() | 각 데이터 요소에 함수를 적용해서 list를 반환 | reduce(fn, data)

* 일반적인 함수로 처리하는 경우

In [28]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c):
    f=list()
    for i in c:
        _f=(float(9)/5)*i + 32
        f.append(_f)
    return f

print c2f(celsius)

[102.56, 97.7, 99.14, 100.03999999999999]


* map()을 사용하는 경우
    * map에 인자로 넘겨주는 함수는 return을 사용한다.

In [8]:
celsius = [39.2, 36.5, 37.3, 37.8]

def c2f(c):
    return (float(9)/5)*c + 32

f=map(c2f, celsius)
print f

[102.56, 97.7, 99.14, 100.03999999999999]


* lambda함수를 사용하는 경우
    * lambda는 무명의 함수

In [9]:
map(lambda c:(float(9)/5)*c + 32, celsius)

[102.56, 97.7, 99.14, 100.03999999999999]

* 문자열에 map()을 적용하는 경우

In [14]:
sentence = 'Hello World'
words = sentence.split()
print words

['Hello', 'World']


* 문자열을 사용하면, 각 단어를 split()한다.
* list를 사용하면, 각 요소를 split()한다.

In [17]:
sentence = "Hello World"
map(lambda x:x.split(),sentence)

[['H'], ['e'], ['l'], ['l'], ['o'], [], ['W'], ['o'], ['r'], ['l'], ['d']]

In [18]:
sentence = ["Hello World"]
map(lambda x:x.split(),sentence)

[['Hello', 'World']]

* filter()

In [37]:
fib = [0,1,1,2,3,5,8,13,21,34,55]
result = filter(lambda x: x % 2, fib)
print result

[1, 1, 3, 5, 13, 21, 55]


* reduce()

In [36]:
reduce(lambda x, y: x+y, range(1,101))

5050

### RDD

* 3단계
    * 1단계: 읽기 - 2가지 방식: 
        * 외부에서 읽기 
        ```
        sc.textFile()
        ```
        
        * 내부에서 읽기 parallelizing a collection
        ```
        sc.parallelize()
        ```
        
    * 2단계: 변환 transformations - lazy도 가능: RDD => RDD or seq(RDD)

함수 | 설명 | 예제
-------|-------|-------
map(fn) | 요소별로 fn을 적용해서 결과 rdd 돌려줌 | map
filter(fn) | 요소별로 선별하여 fn을 적용해서 결과 rdd 돌려줌 | filter(lambda line: "Spark" in line)
flatMap(fn) | 요소별로 fn을 적용하고, flat해서 결과 rdd 돌려줌 | .flatMap(lambda x: x.split(' '))
groupByKey() | |


    * 3단계: actions: RDD => a value (e.g., python list)

함수 | 설명 | 예제
-------|-------|-------
reduce(fn) | 요소별로 fn을 사용해서 줄여서 결과 list를 돌려줌 |
collect() | 모든 요소를 결과 list로 돌려줌 |
count() | 요소의 갯수를 결과 list로 돌려줌 |
countByKey() | |
foreach(fn) | |


* saving
```
rddOfStrings.saveAsTextFile("out.txt")
```

In [3]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Writing data/ds_spark_wiki.txt


* 파일에서 읽기
    ```
    textFile()
    ```

In [4]:
textFile = sc.textFile("data/ds_spark_wiki.txt")

In [5]:
textFile.first()

u'Wikipedia'

* map()함수로 단어 분리하기

In [15]:
words=textFile.map(lambda x:x.split(' '))

In [16]:
words.collect()

[[u'Wikipedia'],
 [u'Apache',
  u'Spark',
  u'is',
  u'an',
  u'open',
  u'source',
  u'cluster',
  u'computing',
  u'framework.'],
 [u'\uc544\ud30c\uce58',
  u'\uc2a4\ud30c\ud06c\ub294',
  u'\uc624\ud508',
  u'\uc18c\uc2a4',
  u'\ud074\ub7ec\uc2a4\ud130',
  u'\ucef4\ud4e8\ud305',
  u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.'],
 [u'Originally',
  u'developed',
  u'at',
  u'the',
  u'University',
  u'of',
  u'California,',
  u"Berkeley's",
  u'AMPLab,'],
 [u'the',
  u'Spark',
  u'codebase',
  u'was',
  u'later',
  u'donated',
  u'to',
  u'the',
  u'Apache',
  u'Software',
  u'Foundation,'],
 [u'which', u'has', u'maintained', u'it', u'since.'],
 [u'Spark',
  u'provides',
  u'an',
  u'interface',
  u'for',
  u'programming',
  u'entire',
  u'clusters',
  u'with'],
 [u'implicit', u'data', u'parallelism', u'and', u'fault-tolerance.']]

* 각 문장의 철자 갯수를 센다.
    * 첫 문장 'Wikipedia'는 9

In [24]:
textFile.map(lambda s:len(s)).collect()

[9, 59, 32, 72, 71, 30, 64, 46]

* filter()

In [6]:
_sparkLine=textFile.filter(lambda line: "Spark" in line)

In [25]:
print _sparkLine.count()

3


* 한글은 앞에 u를 붙여준다.

In [26]:
_line = textFile.filter(lambda line: u"스파크" in line)

In [27]:
print _line.first()

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


* parallelize() 사용하기
    * list에서 읽어, rdd로 변환하기

In [28]:
_aList=[1,2,3]
rdd = sc.parallelize(_aList)

In [44]:
rdd.take(3)

[1, 2, 3]

* map(), collect() 사용해서 square

In [29]:
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
print squared

[1, 4, 9, 16]


* 문장 처리하기
* 단어를 교체하기

In [23]:
a=["this is","a line"]
_rdd=sc.parallelize(a)

words=_rdd.map(lambda x:x.split())
print words.collect()

[['this', 'is'], ['a', 'line']]


In [18]:
_upper=_rdd.map(lambda x:x.replace("a","AA"))
_upper.take(10)

['this is', 'AA line']

* 첫 글자를 대문자로 만들어서 출력해 보기

In [29]:
's'.upper()

'S'

In [30]:
pluralRDD =words.map(lambda x: x[0].upper())
print pluralRDD.collect()

['THIS', 'A']


In [32]:
pluralRDD =words.map(lambda x: [i.upper() for i in x])
print pluralRDD.collect()

[['THIS', 'IS'], ['A', 'LINE']]


* transformation(map()), action(collect()) 함수를 한꺼번에

In [33]:
pluralRDD =words.map(lambda x: [i.upper() for i in x]).collect()
print pluralRDD

[['THIS', 'IS'], ['A', 'LINE']]


In [34]:
wordsLength = words\
    .map(len)\
    .collect()
print wordsLength

[2, 2]


* create RDD from CSV


In [6]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2


Writing ./data/ds_spark_2cols.csv


In [6]:
inp_file = sc.textFile("./data/ds_spark_2cols.csv")
numbers_rdd = inp_file.map(lambda line: line.split(','))

In [7]:
numbers_rdd.take(10)

[[u'35', u' 2'],
 [u'40', u' 27'],
 [u'12', u' 38'],
 [u'15', u' 31'],
 [u'21', u' 1'],
 [u'14', u' 19'],
 [u'46', u' 1'],
 [u'10', u' 34'],
 [u'28', u' 3'],
 [u'48', u' 1']]

* fivethirtyeight
    * git clone https://github.com/fivethirtyeight/uber-tlc-foil-response.git
        daily Uber trip statistics in January and February 2015
        ```
        dispatching_base_number	date	active_vehicles	trips
        B02512	1/1/2015	190	1132
        B02765	1/1/2015	225	1765
        ```


In [4]:
data_home=os.path.join(home,"Code/git/else/uber-tlc-foil-response")
filePath=os.path.join(data_home,"Uber-Jan-Feb-FOIL.csv")

In [5]:
_fub = sc.textFile(filePath)

In [6]:
type(_fub)

pyspark.rdd.RDD

In [29]:
_fub.count()

355

In [30]:
_fub.first()

u'dispatching_base_number,date,active_vehicles,trips'

* csv는 comma seperated 형식이므로, ','로 분리
* 첫번째 열에서 key값을 추출한다 (header값 포함)

In [7]:
_dub = _fub.map(lambda line: line.split(","))

In [8]:
type(_dub)

pyspark.rdd.PipelinedRDD

In [33]:
_row0keys=_dub.map(lambda row: row[0]).distinct().collect()

In [34]:
print _row0keys

[u'B02617', u'B02682', u'B02598', u'B02765', u'B02512', u'dispatching_base_number', u'B02764']


In [15]:
_dub.filter(lambda row: "B02512" in row).count()

59

* B02512인 경우, trips가 2000보다 큰 레코드 수집

In [36]:
_dub.filter(lambda row: "B02512" in row).filter(lambda row: int(row[3])>2000).collect()

[[u'B02512', u'1/30/2015', u'256', u'2016'],
 [u'B02512', u'2/5/2015', u'264', u'2022'],
 [u'B02512', u'2/12/2015', u'269', u'2092'],
 [u'B02512', u'2/13/2015', u'281', u'2408'],
 [u'B02512', u'2/14/2015', u'236', u'2055'],
 [u'B02512', u'2/19/2015', u'250', u'2120'],
 [u'B02512', u'2/20/2015', u'272', u'2380'],
 [u'B02512', u'2/21/2015', u'238', u'2149'],
 [u'B02512', u'2/27/2015', u'272', u'2056']]

* header는 속성 명을 가지고 있다. 이를 제외하면 전체 갯수에서 1개를 뺀 숫자

In [41]:
_noheader = _fub.filter(lambda line: "base" not in line).map(lambda line:line.split(","))
_noheader.count()

354

* reduceByKey - key별로 value를 합쳐서 결과 -> 아래는 a,3 b,2
```
("a", 1)
("b", 1)
("a", 1)
("a", 1)
("b", 1)
```

In [43]:
_noheader.map(lambda x: (x[0], int(x[3]))).reduceByKey(lambda k,v: k + v).collect()

[(u'B02617', 725025),
 (u'B02682', 662509),
 (u'B02598', 540791),
 (u'B02765', 193670),
 (u'B02512', 93786),
 (u'B02764', 1914449)]

## 문제 S-4: word count

* RDD API를 사용해서 단어를 셀 수 있다 (map, reduce 등).
* DataFrame API를 사용하려면 MLib Pipleline을 사용한다.

In [11]:
!ls data/ds_spark_wiki.txt

data/ds_spark_wiki.txt


### line by line

In [ ]:
def mySplit(x):
    return x.split(" ")

words=textFile.map(mySplit)

In [ ]:
type(words)

In [11]:
for i in words.collect():
    print i

* lambda함수를 사용할 수 있다.

In [ ]:
words=textFile.map(lambda x:x.split(' '))

### 전체를 묶어서 word count

In [8]:
lines = sc.textFile("data/ds_spark_wiki.txt")
word_count_bo = lines\
    .flatMap(lambda x: x.split(' '))

In [9]:
word_count_bo.collect()

[u'Wikipedia',
 u'Apache',
 u'Spark',
 u'is',
 u'an',
 u'open',
 u'source',
 u'cluster',
 u'computing',
 u'framework.',
 u'\uc544\ud30c\uce58',
 u'\uc2a4\ud30c\ud06c\ub294',
 u'\uc624\ud508',
 u'\uc18c\uc2a4',
 u'\ud074\ub7ec\uc2a4\ud130',
 u'\ucef4\ud4e8\ud305',
 u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.',
 u'Originally',
 u'developed',
 u'at',
 u'the',
 u'University',
 u'of',
 u'California,',
 u"Berkeley's",
 u'AMPLab,',
 u'the',
 u'Spark',
 u'codebase',
 u'was',
 u'later',
 u'donated',
 u'to',
 u'the',
 u'Apache',
 u'Software',
 u'Foundation,',
 u'which',
 u'has',
 u'maintained',
 u'it',
 u'since.',
 u'Spark',
 u'provides',
 u'an',
 u'interface',
 u'for',
 u'programming',
 u'entire',
 u'clusters',
 u'with',
 u'implicit',
 u'data',
 u'parallelism',
 u'and',
 u'fault-tolerance.']

* 단어를 세어서 tuple로 만듦

In [12]:
from operator import add
lines = sc.textFile("data/ds_spark_wiki.txt")
word_count_bo = lines\
    .flatMap(lambda x: x.split(' '))\
    .map(lambda x: (x.lower().rstrip().lstrip().rstrip(',').rstrip('.'), 1))\
    .reduceByKey(add)

In [13]:
word_count_bo.count()

50

In [14]:
word_count_bo.first()

(u'and', 1)

* unicode

In [11]:
for x in word_count_bo.take(30):
    print x

(u'and', 1)
(u'\uc18c\uc2a4', 1)
(u'is', 1)
(u'interface', 1)
(u'maintained', 1)
(u'donated', 1)
(u'\ucef4\ud4e8\ud305', 1)
(u'open', 1)
(u'for', 1)
(u'\ud074\ub7ec\uc2a4\ud130', 1)
(u'with', 1)
(u'wikipedia', 1)
(u'provides', 1)
(u'was', 1)
(u'which', 1)
(u'codebase', 1)
(u'california', 1)
(u'apache', 2)
(u'spark', 3)
(u'data', 1)
(u'\uc624\ud508', 1)
(u'university', 1)
(u'programming', 1)
(u'\uc2a4\ud30c\ud06c\ub294', 1)
(u'the', 3)
(u'originally', 1)
(u'computing', 1)
(u'developed', 1)
(u'it', 1)
(u'an', 2)


## 문제 S-5: RDD from url

* kaggle
    * https://www.kaggle.com/kaggle/us-baby-names

In [18]:
import os
_datadir='data'
datapath = os.path.join(os.getcwd(),_datadir)
print datapath

/home/jsl/Code/git/bb/jsl/pyds/data


In [19]:
_url = 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'

_fname=os.path.join(datapath,'kddcup.data_10_percent.gz')
if(not os.path.exists(_fname)):
    print "%s data does not exist! retrieving.." % _fname
    _f=urllib.urlretrieve(_url,_fname)


In [20]:
print _fname

/home/jsl/Code/git/bb/jsl/pyds/data/kddcup.data_10_percent.gz


In [21]:
_data = sc.textFile(_fname)

In [22]:
_data.count()

494021

* 데이터에 'normal.'이 포함된 건수

In [20]:
_data.take(5)

[u'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 u'0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 u'0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 u'0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 u'0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [19]:
_normal_data = _data.filter(lambda x: 'normal.' in x)
print _normal_data.count()

97278


In [25]:
_split_data=_data.map(lambda x: x.split(','))
print _split_data.take(5)

[[u'0', u'tcp', u'http', u'SF', u'181', u'5450', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'9', u'9', u'1.00', u'0.00', u'0.11', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.'], [u'0', u'tcp', u'http', u'SF', u'239', u'486', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'19', u'19', u'1.00', u'0.00', u'0.05', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.'], [u'0', u'tcp', u'http', u'SF', u'235', u'1337', u'0', u'0', u'0', u'0', u'0', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'0', u'8', u'8', u'0.00', u'0.00', u'0.00', u'0.00', u'1.00', u'0.00', u'0.00', u'29', u'29', u'1.00', u'0.00', u'0.03', u'0.00', u'0.00', u'0.00', u'0.00', u'0.00', u'normal.'], [u'0', u'tcp', u'http', u'SF', u'219', u'1337', 

* rdd to sql, dataframe

In [23]:
from pyspark.sql import Row

_csv_data = _data.map(lambda l: l.split(","))
_row_data = _csv_data.map(lambda p: 
    Row(
        duration=int(p[0]), 
        protocol_type=p[1],
        service=p[2],
        flag=p[3],
        src_bytes=int(p[4]),
        dst_bytes=int(p[5])
    )
)

In [10]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

_df=sqlCtx.createDataFrame(_row_data)
_df.registerTempTable("_my")

In [ ]:
_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()

In [ ]:
_df.select("protocol_type", "duration", "dst_bytes").filter(_df.duration>1000).filter(_df.dst_bytes==0).groupBy("protocol_type").count().show()

In [11]:
tcp_interactions = sqlCtx.sql("""
    SELECT duration, dst_bytes FROM _my WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [ ]:
tcp_interactions_out = tcp_interactions.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print ti_out

## 문제 S-8: Hello Statistics

* mllib.stat.Statistics

In [37]:
from pyspark.mllib.stat import Statistics

parallelData = sc.parallelize([1.0, 2.0, 5.0, 4.0, 3.0, 3.3, 5.5])

# run a KS test for the sample versus a standard normal distribution
testResult = Statistics.kolmogorovSmirnovTest(parallelData, "norm", 0, 1)
print(testResult)

Kolmogorov-Smirnov test summary:
degrees of freedom = 0 
statistic = 0.841344746068543 
pValue = 5.06089025353873E-6 
Very strong presumption against null hypothesis: Sample follows theoretical distribution.


In [36]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [37]:
from pyspark.sql.functions import rand, randn
 # Create a DataFrame with one int column and 10 rows.
df = sqlCtx.range(0, 10)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [38]:
df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()
df.describe().show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.41371264720975787|  0.5888539012978773|
|  1| 0.7311719281896606|  0.8645537008427937|
|  2| 0.1982919638208397| 0.06157382353970104|
|  3|0.12714181165849525|  0.3623040918178586|
|  4| 0.7604318153406678|-0.49575204523675975|
|  5|0.12030715258495939|  1.0854146699817222|
|  6|0.12131363910425985| -0.5284523629183004|
|  7|0.44292918521277047| -0.4798519469521663|
|  8| 0.8898784253886249| -0.8820294772950535|
|  9|0.03650707717266999| -2.1591956435415334|
+---+-------------------+--------------------+

+-------+------------------+
|summary|                id|
+-------+------------------+
|  count|                10|
|   mean|               4.5|
| stddev|3.0276503540974917|
|    min|                 0|
|    max|                 9|
+-------+------------------+



In [39]:
df = sqlCtx.range(0, 10).withColumn('rand1', rand(seed=10)).withColumn('rand2', rand(seed=27))
print df.stat.corr('rand1', 'rand2')
print df.stat.corr('id', 'id')

0.308427454327
1.0


In [40]:
names = ["Alice", "Bob", "Mike"]
items = ["milk", "bread", "butter", "apples", "oranges"]
df = sqlCtx.createDataFrame([(names[i % 3], items[i % 5]) for i in range(100)], ["name", "item"])
df.show(10)

+-----+-------+
| name|   item|
+-----+-------+
|Alice|   milk|
|  Bob|  bread|
| Mike| butter|
|Alice| apples|
|  Bob|oranges|
| Mike|   milk|
|Alice|  bread|
|  Bob| butter|
| Mike| apples|
|Alice|oranges|
+-----+-------+
only showing top 10 rows



In [42]:
df = sqlCtx.createDataFrame([(1, 2, 3) if i % 2 == 0 else (i, 2 * i, i % 4) for i in range(100)], ["a", "b", "c"])
print df.show(10)
freq = df.stat.freqItems(["a", "b", "c"], 0.4)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
+---+---+---+
only showing top 10 rows

None
